In [81]:
%matplotlib inline
%config InlineBackend.close_figures=False;
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np
from matplotlib import rcParams;
import ipywidgets as widgets;
from ipywidgets import Layout;
from IPython.display import display,clear_output;
import pylab as lp;
from pylab import *;
from scipy.ndimage import measurements;
from IPython.display import display,clear_output;
import warnings;
warnings.filterwarnings('ignore');
import networkx as nx
import random
import os
import scipy.io as sio
from scipy.io import loadmat
from pathlib import Path
import itertools
import collections
import ipywidgets as widgets;
from ipywidgets import Layout
from IPython.display import display,clear_output
from IPython.core.display import display, HTML
from pathlib import Path
from matplotlib import colors
from IPython.display import HTML as html_print
from IPython.display import display



out1 = widgets.Output()
out2 = widgets.Output()

button=widgets.Button(description='Next Spread')
buttonres=widgets.Button(description='Reset')
hbox=widgets.HBox(children=(button,buttonres))
vbox1=widgets.HBox(children=(out1,out2))
vbox=widgets.VBox(children=(vbox1,hbox))
display(vbox)
matplotlib.rcParams['figure.figsize'] = [6, 6]
N=20;
ninf=int(2.5*N);
#############################################
g = nx.grid_2d_graph(N, N);
nng=list(g.nodes());

##############################################
###three distribution-types of infections
##############################################
###1-random infection
#infected=random.sample(nng,ninf)
###end of random infection distrb.
##############################################
###2-diagonal infection
#diagonal = np.arange(N-1,N*N-1,N-1);
#distrib=diagonal;
#infitems=[]
#for i in range(len(distrib)):
#    infitems.append([n for n in nng[distrib[i]]])
#infected=list(tuple(map(tuple, infitems)))
###end of diagonal infection distrb.
##############################################
###3-two-sides infection
xside=np.arange(0,N*N-1,2*N)
yside=np.arange(N*N-N+1,N*N,2)
twosides=np.concatenate((xside,yside), axis=None)
distrib=twosides;
infitems=[]
for i in range(len(distrib)):
    infitems.append([n for n in nng[distrib[i]]])
infected=list(tuple(map(tuple, infitems)))
###end of two-sides infection distrb.
##############################################
def ismember(A,B):
    U, ixU = np.unique(A , return_index=True);
    V = np.in1d(U, B, assume_unique=True);
    ismember = V*1;
    return ismember;
##############################################
step=0;
def spreading(b):
    global step;    
    step +=1; 
    M=[]
    if Path('G_inf.gpickle').is_file():
        G_inf = nx.read_gpickle("G_inf.gpickle")
    else:
        G_inf=nx.Graph()
        G_inf.add_nodes_from(infected)
    inf_nodes=G_inf.nodes();
    Ninf=list(inf_nodes)
    NbInf=len(Ninf);
    inf_array=np.asarray(Ninf)
    
    arrayinf=np.array([list(inf_array[:,0]),list(inf_array[:,1])])
     
    inf_index=np.ravel_multi_index(arrayinf, (N,N))
    lix=np.arange(N*N)
    izInf=ismember(lix,inf_index)
    inf_nodes_inmat0=izInf*(1+lix)
    inf_nodes_inmat1=inf_nodes_inmat0.reshape(N,N)
    inf_nodes_inmat=np.where(inf_nodes_inmat1>0,1,0)
        
    Ninf=list(G_inf.nodes())
    NbInf=len(Ninf);
    
    neighb=[]
    for i in range(len(Ninf)):
        neighb.append([n for n in g.neighbors(Ninf[i])])
    
    allneighbors0=list(itertools.chain.from_iterable(neighb));
    allneighbors= [i for i in allneighbors0 if i not in Ninf ];
    Nb_allneighbors= len(allneighbors)
    neighb2times=[item for item,
                 count in collections.Counter(allneighbors).items()
                 if count > 1]
    NodesContam = [i for i in neighb2times if i not in Ninf ]   
    perim=Nb_allneighbors-len(NodesContam);

    b=.7
    if len(neighb2times)>0:
        cont_array=np.asarray(NodesContam)
        cmap = colors.ListedColormap(['white','black',[b,b,b]])
    else:
        cont_array=inf_array
        cmap = colors.ListedColormap(['cyan',[b,b,b]])
    arraycont=np.array([list(cont_array[:,0]),list(cont_array[:,1])]);
    cont_index=np.ravel_multi_index(arraycont, (N,N));
    lix=np.arange(N*N)

    izCont=ismember(lix,cont_index)
    cont_nodes_inmat0=izCont*(1+lix)
    cont_nodes_inmat1=cont_nodes_inmat0.reshape(N,N)
    cont_nodes_inmat=np.where(cont_nodes_inmat1>0,2,0)
    
    with out1:
        fig1, axes1 = plt.subplots();
        img=inf_nodes_inmat.transpose()+cont_nodes_inmat.transpose()
        Ninf=list(inf_nodes)
        NbInf=len(Ninf);
        im = axes1.imshow(img, cmap);
        axes1.set_xticks(np.arange(-.5, N, 1));
        axes1.set_yticks(np.arange(-.5, N, 1));
        axes1.xaxis.set_tick_params(width=0);
        axes1.yaxis.set_tick_params(width=0);
        axes1.set_xlim([-1,N]);
        axes1.set_ylim([-1,N]);
        axes1.grid(linestyle="-", color='black');
        axes1.set_yticklabels([])
        axes1.set_xticklabels([]) 
        plt.title('pemimeter= '+str(perim)+', Nb. of inf. squ.= '+str(NbInf)+
                  ', Nb. of cont. squ.= '+str(len(NodesContam)))
        plt.show(fig1)

        clear_output(wait=True)
    
    for i in range(len(neighb2times)):
        Ninf.append(tuple([n for n in neighb2times[i]]))
        G_inf.add_nodes_from(Ninf)
        nx.write_gpickle(G_inf,"G_inf.gpickle")

    Ninf=list(G_inf.nodes());
    NbInf2=len(Ninf);
    perim2=Nb_allneighbors-len(NodesContam);
    Nbcont2= len(NodesContam)
    Nbcont= len(NodesContam)

    
    if Path('data_saver.mat').is_file():
        datamat=loadmat('data_saver.mat')
        perim0=datamat['perim']
        perim=perim0[0]
        perim=np.concatenate((perim,perim2), axis=None)
        NbInf0=datamat['NbInf']
        NbInf=NbInf0[0]
        NbInf=np.concatenate((NbInf,NbInf2), axis=None)
        Nbcont0=datamat['Nbcont']
        Nbcont=Nbcont0[0]
        Nbcont=np.concatenate((Nbcont,Nbcont2), axis=None)
        sio.savemat('data_saver.mat', {'NbInf':NbInf,'Nbcont':Nbcont,'perim':perim})
    else:
        sio.savemat('data_saver.mat', {'NbInf':NbInf,'Nbcont':Nbcont,'perim':perim})
        

    datamat=loadmat('data_saver.mat')
    perim0=datamat['perim']
    perim=perim0[0]
    NbInf0=datamat['NbInf']
    NbInf=NbInf0[0]
    xx=np.arange(len(perim))
 
    with out2:
        fig2, axes2 = plt.subplots()
        clear_output(wait=True)
        axes2.plot(xx,NbInf/(N*N),'ro-');
        axes2.plot(xx,10*Nbcont/(N*N),'go-');
        axes2.plot(xx,(N*N-NbInf)/(N*N),'bo-');#(perim[0])
        axes2.set_xticks(np.arange(-0.2, 3*N, N/2));
        axes2.set_yticks(np.arange(-0.2, 3, .2));
        axes2.set_xlim([-.2,3*N]);
        axes2.set_ylim([-.2,3]);
        axes2.grid();
        plt.title('Spreading functions: healthy(b)/infected (r)/contamin.(g)');
        axes2.set_xlabel('Contamination step');
        axes2.set_ylabel('NbInf, perim');

        plt.show(fig2)
        clear_output(wait=True)
        
    
button.on_click(spreading);
spreading(None);

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def print_color(t):
    display(html_print(' '.join([cstr(ti, color=ci) for ti,ci in t])))
    

def reset(b):
    if os.path.exists("G_inf.gpickle"):
        print_color((('To run another Spread, click Reset', 'red'),))
        os.remove("G_inf.gpickle")
        os.remove("data_saver.mat")
        clear_output(wait=True)
    else:
        print_color((('Click the menu Run Button', 'green'),))

buttonres.on_click(reset)